<a href="https://colab.research.google.com/github/TristanShoemaker/COMP551-P3/blob/master/InceptionV3_full_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
# from tensorflow.python.keras._impl import keras

from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.layers import Input
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

import pickle
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

Using TensorFlow backend.


In [0]:
X = pd.read_pickle('train_max_x')
X = X.astype('float32')/255
X = np.repeat(X.reshape(X.shape[0], 128, 128, 1), 3, axis=3)
y = pd.read_csv('train_max_y.csv')
y = y['Label']
y = to_categorical(y, 10)

print(X.shape)
print(y.shape)

(50000, 128, 128, 3)
(50000, 10)


In [0]:
# base model Inception V3
input_tensor = Input(shape=(128, 128, 3)) 
base = InceptionV3(weights='imagenet', include_top=False, input_tensor=input_tensor)

#Add layers
x = base.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
#output layer
predictions = Dense(10, activation='softmax')(x)

#initialize model
model = Model(inputs=input_tensor, outputs=predictions)
len(model.layers)














311


In [0]:
#preprocessing step
def background_removal(image):
  image[image<1-50/255] = 0
  return image

In [0]:
#print images
def show(image):
    plt.figure(figsize=(6,5))
    plt.imshow(image, label="", cmap='gray', vmin=0, vmax=1)
    plt.colorbar()

In [0]:
#rerun several times, load and save weights into modelweights file

model.load_weights("./modelweights.h5")

#Train only the last layers (first 310 are from Inception V3) (leave commented to train the entire layer)
#for layer in model.layers[:310]:
 # layer.trainable = False

#compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])


#train
batch_size = 500
img = ImageDataGenerator(rotation_range=180, width_shift_range=int(20), height_shift_range=int(20), zoom_range=0.2, preprocessing_function=bg_rm_rand, validation_split=0.15)
training_data = img.flow(X, y, batch_size=batch_size, subset='training')
validation_data = img.flow(X, y, batch_size=batch_size, subset='validation')

fit = model.fit_generator(training_data, epochs=10, verbose=1, validation_data=validation_data)


model.save_weights("./modelweights.h5")

49
Epoch 1/10
49/49 [==============================] - 629s 13s/step - loss: 0.0270 - categorical_accuracy: 0.9933
Epoch 2/10
49/49 [==============================] - 613s 13s/step - loss: 0.0088 - categorical_accuracy: 0.9976
Epoch 3/10
49/49 [==============================] - 602s 12s/step - loss: 0.0050 - categorical_accuracy: 0.9985
Epoch 4/10


KeyboardInterrupt: ignored